In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

jokes = pd.read_csv('jester-data-1.csv')
print('Number of Ratings:'+str(len(jokes)))
print(jokes.head(10))


Number of Ratings:24983
   Joke1  Joke2  Joke3  Joke4  Joke5  ...  Joke96  Joke97  Joke98  Joke99  Joke100
0  -7.82   8.79  -9.66  -8.16  -7.52  ...   99.00   -5.63   99.00   99.00    99.00
1   4.08  -0.29   6.36   4.37  -2.38  ...   -2.14    3.06    0.34   -4.32     1.07
2  99.00  99.00  99.00  99.00   9.03  ...   99.00   99.00   99.00   99.00    99.00
3  99.00   8.35  99.00  99.00   1.80  ...   99.00   99.00   99.00   99.00    99.00
4   8.50   4.61  -4.17  -5.39   1.36  ...    1.55    3.11    6.55    1.80     1.60
5  -6.17  -3.54   0.44  -8.50  -7.09  ...   -3.35    0.05   -9.08   -5.05    -3.45
6  99.00  99.00  99.00  99.00   8.59  ...    2.33   99.00   99.00   99.00    99.00
7   6.84   3.16   9.17  -6.21  -8.16  ...   -3.35    2.14   -0.05    1.31     0.00
8  -3.79  -3.54  -9.42  -6.89  -8.74  ...   -0.29   -0.29   -0.29   -3.40    -4.95
9   3.01   5.15   5.15   3.01   6.41  ...   99.00   99.00   99.00   99.00    99.00

[10 rows x 100 columns]


In [ ]:
from random import randrange
number_to_remove = 625000 

sparse_jokes = np.array(jokes, copy=True)
initial_count_99 = np.sum(np.count_nonzero(jokes == 99, axis=0))
print("Initial count", initial_count_99)

np.random.seed(32) # TODO : remove this
m = np.random.randint(low=0, high=len(jokes), size=(len(jokes),))
p = np.random.randint(low=0, high=jokes.shape[1], size=(jokes.shape[1],))
list_indices=[]

count = 0 
for row_index in m:
  for column_index in p :
    if sparse_jokes[row_index][column_index] != 99:

      sparse_jokes[row_index][column_index] = 99
      count +=1
      list_indices.append((row_index,column_index))
    if count == number_to_remove:
      break
  else:
      continue  # only executed if the inner loop did NOT break
  break



final_count_99 = np.sum(np.count_nonzero(sparse_jokes == 99, axis=0))
print("Final count", final_count_99)

Initial count 687845
Final count 1312845


In [ ]:
#scaling negative data for non negative matrix factorization
jokes_array_sparse_scaled = np.array(sparse_jokes)
n = jokes_array_sparse_scaled.shape[0]
m = jokes_array_sparse_scaled.shape[1]
for i in range(n):
  for j in range(m):
    if(jokes_array_sparse_scaled[i][j]!=99):
      jokes_array_sparse_scaled[i][j] = jokes_array_sparse_scaled[i][j] + 10

In [ ]:
##### Prepare dense dataset  ######
 
present_99_sparse = np.sum(np.count_nonzero(jokes_array_sparse_scaled == 99, axis=0))
print(present_99_sparse)

sparse_jokes_clean = np.copy(jokes_array_sparse_scaled)
rows_with_99 = np.count_nonzero(sparse_jokes_clean == 99, axis=1)
rows_with_99_index= np.where(rows_with_99 != 0)


dense_matrix = np.delete(sparse_jokes_clean, rows_with_99_index, axis=0 )

present_99_dense = np.sum(np.count_nonzero(dense_matrix == 99, axis=0))
print("Dense matrix 99 count", present_99_dense)


1312845
Dense matrix 99 count 0


In [ ]:
def baseline_rating_i(ratings):
        ratings = ratings.ravel()
        ratings_non_99 = ratings[ratings != 99]
        #return  np.sum(ratings_non_99)/ratings_non_99.shape[0]
        return np.mean(ratings_non_99)

In [ ]:
def baseline_rating_j(ratings):
        #ratings = ratings.ravel()
        ratings_non_99 = ratings[ratings != 99]
        #return  np.sum(ratings_non_99)/ratings_non_99.shape[0]
        return np.mean(ratings_non_99,axis=0)

In [ ]:
from scipy import stats
def pearson_similarity(dense_matrix,joke_index_to_predict=1):
    dense_jokes_pearson = np.copy(dense_matrix)


    primary_joke_pearson = dense_jokes_pearson[:,joke_index_to_predict].reshape(dense_jokes_pearson.shape[0], -1)
    secondary_jokes = dense_jokes_pearson

    similarities = []
    for i in range(secondary_jokes.shape[1]):
      pearson_similarity, _ = stats.pearsonr(primary_joke_pearson.ravel(), secondary_jokes[:,i].ravel())
      similarities.append(pearson_similarity)

    similarities = np.array(similarities)
    
    pearson_similarity_value = np.sort(similarities)[::-1]

    similarities = np.argsort(similarities)[::-1]
    return similarities, pearson_similarity_value

In [ ]:
##### Soft KNN with User baseline adjusted ######
def compute_soft_knn_user_baseline_pearson(k=99):
  jokes_sim_pearson = {}
  jokes_sim_pearson_value = {}
  for r in range (0,jokes_array_sparse_scaled.shape[1]):
      jokes_sim_pearson[r] , jokes_sim_pearson_value[r] = pearson_similarity(dense_matrix,r)

  user_indices=jokes_array_sparse_scaled.shape[0]
  joke_indices=jokes_array_sparse_scaled.shape[1]


  sparse_matrix_knn_user_baseline_pearson = np.copy(jokes_array_sparse_scaled)
  joke_baseline_rating_j = {}
  joke_baseline_rating_i = {}

  for r in range (0,100):
      joke_baseline_rating_j[r] = baseline_rating_j(sparse_matrix_knn_user_baseline_pearson[:,jokes_sim_pearson[r][:k]])

  for r in range (0,100):
      joke_baseline_rating_i[r] = baseline_rating_i(sparse_matrix_knn_user_baseline_pearson[:,r])

  for user in range(user_indices):
    # user_baseline_rating_i = baseline_rating(sparse_matrix_knn_user_baseline_cosine[user])
    # print("baseline :: ", user_baseline_rating)
    for joke in range(joke_indices):
      # user_baseline_rating_i = baseline_rating(sparse_matrix_knn_user_baseline_cosine[:])
      if sparse_matrix_knn_user_baseline_pearson[user][joke]==99:
        similar_jokes=jokes_sim_pearson[joke]
        similar_jokes_pearson_value = jokes_sim_pearson_value[joke]

        count=0
        sum=0
        joke_num=1

        # Check how many similar jokes the user has rated
        user_data = np.copy(sparse_matrix_knn_user_baseline_pearson[user])
        user_data_sorted = user_data[similar_jokes]

        ##Finding the similarity values from other users for those jokes which the current user has rated
        similarity_values_user_ratings = similar_jokes_pearson_value[user_data_sorted != 99]
        similarity_values_user_ratings_k = similarity_values_user_ratings[:k]

        user_non_99 = user_data_sorted[user_data_sorted != 99]
        user_non_99_k = user_non_99[:k]
        if (user_non_99_k.shape[0]):
          # user rated similar jokes found
          sparse_matrix_knn_user_baseline_pearson[user][joke] = joke_baseline_rating_i[joke] +(np.sum(similarity_values_user_ratings_k*(user_non_99_k-joke_baseline_rating_j[joke]))/np.sum(similarity_values_user_ratings_k))
        else:
          # No user rated similar jokes found
          sparse_matrix_knn_user_baseline_pearson[user][joke]=jokes_mean_imputed[user][joke]

  return sparse_matrix_knn_user_baseline_pearson

In [ ]:
# Performs KNN_Cosine similarity for imputing ratings and loss for different K values 
k=[10]
count=625000
#k=[2]
l1_losses_KNN_cosine_user_baseline=[]
l2_losses_KNN_cosine_user_baseline=[]

for i in k:
  print("k=",i)
  sparse_matrix_knn_user_baseline_cosine=compute_soft_knn_user_baseline_pearson(i)

k= 10


In [ ]:
######### NNMF ############

In [ ]:
#scaling negative data for non negative matrix factorization
jokes_array = np.array(jokes)
n = jokes_array.shape[0]
m = jokes_array.shape[1]
for i in range(n):
  for j in range(m):
    if(jokes_array[i][j]!=99):
      jokes_array[i][j] = jokes_array[i][j] + 10

In [ ]:
#replace missing values with mean
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error

mean_simple_imputer = SimpleImputer(missing_values=99, strategy='mean')
jokes_mean_imputed = mean_simple_imputer.fit_transform(jokes_array)

no_of_99_jokes = np.sum(np.count_nonzero(jokes_mean_imputed == 99, axis=0))

In [ ]:
import numpy as np
from sklearn.decomposition import NMF
model = NMF(n_components=2, init='random', random_state=0)
W = model.fit_transform(jokes_mean_imputed)
H = model.components_

/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_nmf.py:1641: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  ConvergenceWarning,


In [ ]:
jokes_predicted = np.dot(W,H)

In [ ]:
## Compare jokes_predicted and sparse_matrix_knn_user_baseline_cosine for the values 
## which were not 99 in original matrix
jokes_array_sparse_scaled_array = np.array(jokes_array_sparse_scaled, copy=True)
jokes_array = np.array(jokes,  copy=True)

In [ ]:
n = jokes_array.shape[0]
m = jokes_array.shape[1]
count = 0

L1_error = 0 
L1_error_user_baseline = 0 
L1_error_NNF = 0 


predicted_ratings = list() # NNMF rating , user-baseline
actual_ratings = list()
for i in range(n):
  for j in range(m):
    if(jokes_array[i][j]!=99 and jokes_array_sparse_scaled_array[i][j] == 99):
      count +=1
      # ensemble = (jokes_predicted[i][j] + sparse_matrix_knn_user_baseline_cosine[i][j])/2

      # L1_error += np.abs(jokes_array_sparse_scaled_array[i][j] - ensemble)
      # L1_error_user_baseline += np.abs(jokes_array_sparse_scaled_array[i][j] - sparse_matrix_knn_user_baseline_cosine[i][j])
      # L1_error_NNF += np.abs(jokes_array_sparse_scaled_array[i][j] - jokes_predicted[i][j])

      predicted_ratings.append([jokes_predicted[i][j], sparse_matrix_knn_user_baseline_cosine[i][j]])
      actual_ratings.append(jokes_array[i][j])

predicted_ratings = np.array(predicted_ratings)
actual_ratings = np.array(actual_ratings)

print(predicted_ratings.shape)
print(actual_ratings.shape)
# print("Ensemble", L1_error, count, L1_error/count)
# print("User baseline", L1_error_user_baseline, count, L1_error_user_baseline/count)
# print("NNMF", L1_error_NNF, count, L1_error_NNF/count)


(625000, 2)
(625000,)


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(predicted_ratings, actual_ratings, test_size=0.20, random_state=42)
reg = LinearRegression().fit(X_train, y_train)

y_test_predict = reg.predict(X_test)

0.38672808008599857


In [ ]:
count = 0
L2_error_ensemble = 0 

for i in range(y_test.shape[0]):
      count +=1

      L2_error_ensemble += np.square(np.abs(y_test[i] - y_test_predict[i]))


print("Ensemble L2 loss : ", L2_error_ensemble/count)

Ensemble L2 loss :  17.05664013316251


In [ ]:
coefficients = reg.coef_
print(coefficients)

[0.49556079 0.55416714]


In [ ]:
### Predicting Ensemble ####
n = jokes_array.shape[0]
m = jokes_array.shape[1]
ensemble_jokes_array = np.copy(jokes_array)

count = 0

L1_error = 0 
L1_error_user_baseline = 0 
L1_error_NNF = 0 

coefficients = reg.coef_

predicted_ratings = list() # NNMF rating , user-baseline
actual_ratings = list()
for i in range(n):
  for j in range(m):
    if(jokes_array_sparse_scaled_array[i][j] == 99):
      predicted_rating = coefficients[0] * jokes_predicted[i][j] + coefficients[1] * sparse_matrix_knn_user_baseline_cosine[i][j]
      jokes_array_sparse_scaled_array[i][j] = predicted_rating


Final count 0


In [ ]:
jokes_values = pd.read_csv('jokeid_to_joke.csv')
jokes_indices = np.array(jokes_values)
jokes_indices = jokes_indices[jokes_indices[:, 0].argsort()]


In [ ]:
def recommendation(final_matrix):
  n = final_matrix.shape[0]
  m = final_matrix.shape[1]
  jokes_indices = np.array(jokes_values)
  recommendation_matrix = np.zeros((n,m), dtype=int)
  for i in range(n):
    recommendation_matrix[i] = np.argsort(final_matrix[i])
    
  print(recommendation_matrix)
  
  number_of_suggestions = 10
  for i in range(n):
    print("For user:")
    print(i)
    for j in range(number_of_suggestions):
      print(jokes_indices[recommendation_matrix[i][j]][1])

In [ ]:
recommendation(ensemble_jokes_array)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
###### ENSEMBLE USING THE AVERAGE #######


n = jokes_array.shape[0]
m = jokes_array.shape[1]
count = 0
L2_error = 0 



for i in range(n):
  for j in range(m):
    if(jokes_array[i][j]!=99 and jokes_array_sparse_scaled_array[i][j] == 99):
      count +=1
      ensemble = (jokes_predicted[i][j] + sparse_matrix_knn_user_baseline_cosine[i][j])/2
      L2_error += np.square(np.abs(jokes_array[i][j] - ensemble))


print("Ensemble L2 loss", L2_error, count, L2_error/count)




Ensemble L2 loss 74642508.44442475 625000 119.4280135110796
